In [16]:
!pip3 install datasets transformers

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import time
import pandas as pd
import numpy as np

In [2]:
!wget https://www.htw-dresden.de/~guhr/dist/sample/germeval2018.training.txt
!wget https://www.htw-dresden.de/~guhr/dist/sample/germeval2018.test.txt

--2020-11-25 16:34:40--  https://www.htw-dresden.de/~guhr/dist/sample/germeval2018.training.txt
Resolving www.htw-dresden.de (www.htw-dresden.de)... 141.56.16.110
Connecting to www.htw-dresden.de (www.htw-dresden.de)|141.56.16.110|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.htw-dresden.de/~guhr/dist/sample/germeval2018.training.txt [following]
--2020-11-25 16:34:40--  https://www2.htw-dresden.de/~guhr/dist/sample/germeval2018.training.txt
Resolving www2.htw-dresden.de (www2.htw-dresden.de)... 141.56.16.23
Connecting to www2.htw-dresden.de (www2.htw-dresden.de)|141.56.16.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 795530 (777K) [text/plain]
Saving to: ‘germeval2018.training.txt’

germeval2018.traini 100%[===================>] 776,88K  --.-KB/s    in 0,02s   

2020-11-25 16:34:40 (33,8 MB/s) - ‘germeval2018.training.txt’ saved [795530/795530]

--2020-11-25 16:34:40--  https://www.htw-dresden.de/~

In [3]:
test_df = pd.read_csv("germeval2018.test.txt", sep='\t', header=0,encoding="utf-8")
train_df = pd.read_csv("germeval2018.training.txt", sep='\t', header=0,encoding="utf-8")

In [4]:
train_df.head()

,text,label,label2
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT


In [5]:
# drop unused columns
test_df.drop(columns=['label2'], inplace=True)
train_df.drop(columns=['label2'], inplace=True)

In [6]:
def clean_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9öäüÖÄÜß()!?]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

def convert_label(label):
    return 1 if label == "OFFENSE" else 0

In [8]:
train_df["text"]=clean_text(train_df["text"])
test_df["text"]=clean_text(test_df["text"])
train_df["label"]=train_df["label"].map(convert_label)
test_df["label"]=test_df["label"].map(convert_label)

In [17]:
# this is  how our data set looks now. No urls no @ :)
train_df.head()

,text,label
0,corinnamilborn liebe corinna wir würden dich g...,0
1,martin28a sie haben ja auch recht unser tweet ...,0
2,ahrens theo fröhlicher gruß aus der schönsten ...,0
3,dushanwegner amis hätten alles und jeden gewäh...,0
4,spdde kein verläßlicher verhandlungspartner na...,1


In [90]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
train_dataset.rename_column_("text", "sentence")
test_dataset = Dataset.from_pandas(test_df)
test_dataset.rename_column_("text", "sentence")

In [ ]:
#todo move cleaning process to dataset type

In [91]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

model_checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [92]:
tokenizer(["Das ist ein Test.", "And this is a second sentence"], truncation=True,padding=True)

{'input_ids': [[101, 8695, 21541, 16417, 3231, 1012, 102, 0], [101, 1998, 2023, 2003, 1037, 2117, 6251, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [106]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True,padding=True)

In [107]:
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

In [108]:
encoded_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'sentence'],
    num_rows: 5009
})

In [109]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [110]:
batch_size = 16

args = TrainingArguments(
    "test-offsive-language",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [111]:
metric = load_metric("accuracy","f1") 

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)

In [112]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [113]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [81]:
encoded_test_dataset[0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  